# IR3 Last column (counts) of the patients for the BL visit.

In [11]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import os
import glob
import functools
from pathlib import Path

In [3]:
#!pip install dask[complete];
#!pip install --upgrade pandas "dask[complete]"
#python -m pip install "dask[dataframe]" --upgrade

In [12]:
path_to_files="/scratch/znazari/PPMI_ver_sep2022/RNA_Seq_data/star_ir3/counts/"
path1=Path("/scratch/znazari/PPMI_ver_sep2022/RNA_Seq_data/star_ir3/counts/")
path2 = Path("/home/znazari/ppmi_files/data_preprocessing/data") # where data will be saved at the end.

Note that the counts file in the IR3 is around 152 G

In [13]:
#reading the files which are in BL (Base line) visit.
specific_word = 'BL'
ending_pattern = '*.txt'
file_pattern = f'*{specific_word}*.{ending_pattern}'
file_paths = glob.glob(path_to_files + file_pattern)

In [ ]:
filename = 'bl.txt'
file_path_2 = os.path.join(path_to_files, filename)
bl_files = pd.read_csv(file_path_2,header=None)

In [ ]:
# We define a function where we can take the second phrase seperated by dot. The second phrase 
# is the patient ID. So with this functin we want to get the patient IDs from their file's name
def function_names(fname):
    tokens=fname.split('.')
    return tokens[1]

In [ ]:
# we create a list with the name of the each patients.
bl_list = [function_names(bl_files.iloc[i][0]) for i in range(len(bl_files))]

In [ ]:
%%time
# here we read all the files with with base visit(BL) from the counts folder (where we have all the files
# for all the patients and all the visit).

list_bl_files = [dd.read_csv(path1/bl_files.iloc[i][0],skiprows=1,delimiter='\t') for i in range(len(bl_files))]

In [ ]:
# we get the Geneid column and convert it to dask dataframe
pd_tmp_file = list_bl_files[3].compute()
geneid = pd_tmp_file['Geneid']
ddf_geneid = dd.from_pandas(geneid, npartitions=1)

In [ ]:
# we get th last columns of each file in the list
last_columns = [ddf.iloc[:, -1:] for ddf in list_bl_files]

In [10]:
single_file = dd.concat(last_columns, axis=1)

/home/znazari/anaconda3/lib/python3.9/site-packages/dask/dataframe/multi.py:1291: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


In [11]:
# we change the name of the each columns with the patient numbers.
single_file.columns = bl_list

In [12]:
# here we set the Geneid column as the index of the matrix.
ddf_new_index = single_file.set_index(ddf_geneid)

In [14]:
%%time
ir3_counts = ddf_new_index.compute()
ir3_counts.to_csv(path2/"matrix_ir3_counts_bl.csv")

CPU times: user 18min 54s, sys: 6min 5s, total: 25min
Wall time: 16min 20s


In [16]:
read_ir3_counts = pd.read_csv(path2/"matrix_ir3_counts_bl.csv")

In [ ]:
read_ir3_counts.set_index('Geneid', inplace=True)

In [30]:
read_ir3_counts

,10874,12499,12593,13039,13424,14281,14331,14426,15761,16580,...,60024,60036,60095,60171,65002,65003,65005,65008,70239,85242
Geneid,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003.14,29,18,32,11,18,20,9,15,23,7,...,20,28,6,27,19,17,44,32,10,5
ENSG00000000005.5,3,0,9,0,0,9,0,0,5,0,...,0,1,0,0,2,2,1,14,0,0
ENSG00000000419.12,877,704,574,820,677,310,863,1252,520,631,...,699,707,550,501,1049,841,1438,431,513,314
ENSG00000000457.13,1686,1781,1704,1682,1656,695,1431,1778,1232,1034,...,1670,1651,1331,1013,1288,1732,2057,710,1275,858
ENSG00000000460.16,574,570,551,527,669,236,423,555,312,347,...,531,571,446,304,431,550,542,300,397,196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000285990.1,1,2,2,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
ENSG00000285991.1,3,2,5,0,8,3,3,0,1,2,...,0,0,1,0,0,3,1,8,2,1
ENSG00000285992.1,7,0,5,1,1,3,0,0,1,0,...,0,1,0,0,0,2,0,3,1,0
